In [ ]:
from IPython.display import clear_output
! pip install alphagenome
clear_output()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls

sample_data


In [ ]:
from io import StringIO
from alphagenome import colab_utils
from alphagenome.data import genome
from alphagenome.models import dna_client, variant_scorers
from google.colab import data_table, files
import pandas as pd
from tqdm import tqdm

data_table.enable_dataframe_formatter()

# Load the model.
dna_model = dna_client.create(colab_utils.get_api_key())

In [ ]:
# @title Score batch of variants.

# # Load VCF file containing variants.
# vcf_file = 'placeholder'  # @param

# # We provide an example list of variants to illustrate.
# vcf_file = """variant_id\tCHROM\tPOS\tREF\tALT
# chr3_58394738_A_T_b38\tchr3\t58394738\tA\tT
# chr8_28520_G_C_b38\tchr8\t28520\tG\tC
# chr16_636337_G_A_b38\tchr16\t636337\tG\tA
# chr16_1135446_G_T_b38\tchr16\t1135446\tG\tT
# """

vcf = pd.read_csv('/content/drive/MyDrive/AlphaGenome/K562_var2grn_pip70_alphagenome_format.tsv', sep='\t')
# vcf = vcf.iloc[1000:]
required_columns = ['variant_id', 'CHROM', 'POS', 'REF', 'ALT']
for column in required_columns:
  if column not in vcf.columns:
    raise ValueError(f'VCF file is missing required column: {column}.')

organism = 'human'  # @param ["human", "mouse"] {type:"string"}

# @markdown Specify length of sequence around variants to predict:
sequence_length = '1MB'  # @param ["2KB", "16KB", "100KB", "500KB", "1MB"] { type:"string" }
sequence_length = dna_client.SUPPORTED_SEQUENCE_LENGTHS[
    f'SEQUENCE_LENGTH_{sequence_length}'
]

In [ ]:
vcf

,variant_id,CHROM,POS,REF,ALT
0,chr6_41957260_C_T_b38,chr6,41957260,C,T
1,chr4_144104973_C_T_b38,chr4,144104973,C,T
2,chr5_88884379_T_C_b38,chr5,88884379,T,C
3,chr11_62042222_T_C_b38,chr11,62042222,T,C
4,chr4_54571325_T_C_b38,chr4,54571325,T,C
5,chr1_231421418_C_G_b38,chr1,231421418,C,G
6,chr8_21960657_T_C_b38,chr8,21960657,T,C
7,chr2_8645073_A_C_b38,chr2,8645073,A,C
8,chr17_78014040_C_T_b38,chr17,78014040,C,T
9,chr5_136082636_C_A_b38,chr5,136082636,C,A


In [ ]:
# @markdown Specify which scorers to use to score your variants:
score_rna_seq = True  # @param { type: "boolean"}
score_cage = True  # @param { type: "boolean" }
score_procap = True  # @param { type: "boolean" }
score_atac = True  # @param { type: "boolean" }
score_dnase = True  # @param { type: "boolean" }
score_chip_histone = True  # @param { type: "boolean" }
score_chip_tf = True  # @param { type: "boolean" }
score_polyadenylation = True  # @param { type: "boolean" }
score_splice_sites = True  # @param { type: "boolean" }
score_splice_site_usage = True  # @param { type: "boolean" }
score_splice_junctions = True  # @param { type: "boolean" }

# @markdown Other settings:
download_predictions = False  # @param { type: "boolean" }

# Parse organism specification.
organism_map = {
    'human': dna_client.Organism.HOMO_SAPIENS,
    'mouse': dna_client.Organism.MUS_MUSCULUS,
}
organism = organism_map[organism]

# Parse scorer specification.
scorer_selections = {
    'rna_seq': score_rna_seq,
    'cage': score_cage,
    'procap': score_procap,
    'atac': score_atac,
    'dnase': score_dnase,
    'chip_histone': score_chip_histone,
    'chip_tf': score_chip_tf,
    'polyadenylation': score_polyadenylation,
    'splice_sites': score_splice_sites,
    'splice_site_usage': score_splice_site_usage,
    'splice_junctions': score_splice_junctions,
}

all_scorers = variant_scorers.RECOMMENDED_VARIANT_SCORERS
selected_scorers = [
    all_scorers[key]
    for key in all_scorers
    if scorer_selections.get(key.lower(), False)
]

# Remove any scorers or output types that are not supported for the chosen organism.
unsupported_scorers = [
    scorer
    for scorer in selected_scorers
    if (
        organism.value
        not in variant_scorers.SUPPORTED_ORGANISMS[scorer.base_variant_scorer]
    )
    | (
        (scorer.requested_output == dna_client.OutputType.PROCAP)
        & (organism == dna_client.Organism.MUS_MUSCULUS)
    )
]
if len(unsupported_scorers) > 0:
  print(
      f'Excluding {unsupported_scorers} scorers as they are not supported for'
      f' {organism}.'
  )
  for unsupported_scorer in unsupported_scorers:
    selected_scorers.remove(unsupported_scorer)


# Score variants in the VCF file.
results = []

for i, vcf_row in tqdm(vcf.iterrows(), total=len(vcf)):
  variant = genome.Variant(
      chromosome=str(vcf_row.CHROM),
      position=int(vcf_row.POS),
      reference_bases=vcf_row.REF,
      alternate_bases=vcf_row.ALT,
      name=vcf_row.variant_id,
  )
  interval = variant.reference_interval.resize(sequence_length)

  variant_scores = dna_model.score_variant(
      interval=interval,
      variant=variant,
      variant_scorers=selected_scorers,
      organism=organism,
  )
  results.append(variant_scores)

df_scores = variant_scorers.tidy_scores(results)

if download_predictions:
  df_scores.to_csv('variant_scores.csv', index=False)
  files.download('variant_scores.csv')

# df_scores

100%|██████████| 30/30 [01:15<00:00,  2.51s/it]


In [ ]:
df_scores['variant_id'].astype(str).nunique(), len(df_scores)

(30, 621982)

In [ ]:
# df_scores['variant_id'].astype(str).nunique()

In [ ]:
df_scores.to_csv('/content/drive/MyDrive/AlphaGenome/alphagenome_score/K562_var2grn_pip70_alphagenome_score.tsv', sep='\t', index=False)

In [ ]:
df_scores['variant_id'] = df_scores['variant_id'].astype(str)

In [ ]:
df_scores# .head()

,variant_id,scored_interval,gene_id,gene_name,gene_type,gene_strand,junction_Start,junction_End,output_type,variant_scorer,...,track_strand,Assay title,ontology_curie,biosample_name,biosample_type,transcription_factor,histone_mark,gtex_tissue,raw_score,quantile_score
0,chr6:41957260:C>T,chr6:41432972-42481548:.,None,None,None,None,None,None,ATAC,"CenterMaskScorer(requested_output=ATAC, width=...",...,.,ATAC-seq,CL:0000084,T-cell,primary_cell,NaN,NaN,NaN,-0.031329,-0.811199
1,chr6:41957260:C>T,chr6:41432972-42481548:.,None,None,None,None,None,None,ATAC,"CenterMaskScorer(requested_output=ATAC, width=...",...,.,ATAC-seq,CL:0000100,motor neuron,in_vitro_differentiated_cells,NaN,NaN,NaN,-0.247378,-0.972376
2,chr6:41957260:C>T,chr6:41432972-42481548:.,None,None,None,None,None,None,ATAC,"CenterMaskScorer(requested_output=ATAC, width=...",...,.,ATAC-seq,CL:0000236,B cell,primary_cell,NaN,NaN,NaN,-0.082275,-0.934881
3,chr6:41957260:C>T,chr6:41432972-42481548:.,None,None,None,None,None,None,ATAC,"CenterMaskScorer(requested_output=ATAC, width=...",...,.,ATAC-seq,CL:0000623,natural killer cell,primary_cell,NaN,NaN,NaN,-0.112340,-0.981678
4,chr6:41957260:C>T,chr6:41432972-42481548:.,None,None,None,None,None,None,ATAC,"CenterMaskScorer(requested_output=ATAC, width=...",...,.,ATAC-seq,CL:0000624,"CD4-positive, alpha-beta T cell",primary_cell,NaN,NaN,NaN,-0.045804,-0.862638
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
621977,chr13:75481466:G>A,chr13:74957178-76005754:.,ENSG00000136111,TBC1D4,protein_coding,-,None,None,RNA_SEQ,PolyadenylationScorer(),...,.,polyA plus RNA-seq,UBERON:0018115,left renal pelvis,tissue,NaN,NaN,,0.003811,0.751654
621978,chr13:75481466:G>A,chr13:74957178-76005754:.,ENSG00000136111,TBC1D4,protein_coding,-,None,None,RNA_SEQ,PolyadenylationScorer(),...,.,polyA plus RNA-seq,UBERON:0018116,right renal pelvis,tissue,NaN,NaN,,0.002566,0.563110
621979,chr13:75481466:G>A,chr13:74957178-76005754:.,ENSG00000136111,TBC1D4,protein_coding,-,None,None,RNA_SEQ,PolyadenylationScorer(),...,.,polyA plus RNA-seq,UBERON:0018117,left renal cortex interstitium,tissue,NaN,NaN,,0.003187,0.686638
621980,chr13:75481466:G>A,chr13:74957178-76005754:.,ENSG00000136111,TBC1D4,protein_coding,-,None,None,RNA_SEQ,PolyadenylationScorer(),...,.,polyA plus RNA-seq,UBERON:0018118,right renal cortex interstitium,tissue,NaN,NaN,,0.004605,0.788366


In [ ]:
# df_scores[df_scores['variant_id'] == 'chr3:58394738:A>T'][['output_type', 'biosample_name', 'raw_score']]

,output_type,biosample_name,raw_score
0,ATAC,T-cell,-0.003778
1,ATAC,motor neuron,0.012303
2,ATAC,B cell,-0.000334
3,ATAC,natural killer cell,-0.009233
4,ATAC,"CD4-positive, alpha-beta T cell",-0.010510
...,...,...,...
20324,RNA_SEQ,left renal pelvis,0.000705
20325,RNA_SEQ,right renal pelvis,0.002350
20326,RNA_SEQ,left renal cortex interstitium,0.001645
20327,RNA_SEQ,right renal cortex interstitium,0.002350


In [ ]:
print(vcf_file)

variant_id	CHROM	POS	REF	ALT
chr3_58394738_A_T_b38	chr3	58394738	A	T
chr8_28520_G_C_b38	chr8	28520	G	C
chr16_636337_G_A_b38	chr16	636337	G	A
chr16_1135446_G_T_b38	chr16	1135446	G	T



In [ ]:
df_scores['variant_id'].unique()

array(['chr3:58394738:A>T', 'chr8:28520:G>C', 'chr16:636337:G>A',
       'chr16:1135446:G>T'], dtype=object)